## การแก้ปัญหาการวางแผน (Solving Planning Problems)
----
### GRAPHPLAN
<br>
อัลกอริทึม GraphPlan เป็นวิธีที่ได้รับความนิยมสำหรับการแก้ปัญหาการวางแผนแบบคลาสสิก (classical planning).
ก่อนที่เราจะลงรายละเอียดของอัลกอริทึม มาดูโครงสร้างข้อมูลพิเศษที่เรียกว่า **planning graph** ซึ่งช่วยให้ได้ค่าประมาณเชิงฮิวริสติกที่ดียิ่งขึ้นและมีบทบาทสำคัญในอัลกอริทึม GraphPlan.

### แผนภาพการวางแผน (Planning Graph)
Planning graph คือกราฟแบบมีทิศทางที่จัดเป็นระดับ (levels)
แต่ละระดับบรรจุข้อมูลเกี่ยวกับสถานะปัจจุบันของฐานความรู้และลิงก์ระหว่างสถานะ-การกระทำที่ไปสู่และมาจากระดับนั้น
ระดับแรกประกอบด้วยสถานะเริ่มต้น โดยมีโหนดแทนแต่ละฟลูเอนต์ (fluent) ที่เป็นจริงในระดับนั้น
ระดับนี้มีลิงก์สถานะ-การกระทำ ที่เชื่อมแต่ละสถานะไปยังการกระทำที่ถูกต้องในสถานะนั้น
การกระทำแต่ละอย่างเชื่อมโยงกับเงื่อนไขก่อน (preconditions) ทั้งหมดและสถานะผลลัพธ์ (effects) ของมัน
โดยอิงจากผลลัพธ์เหล่านี้ จะสร้างระดับถัดไปขึ้นมา
ระดับถัดไปจะมีโครงสร้างข้อมูลลักษณะเดียวกันสำหรับสถานะถัดไป
ด้วยวิธีนี้ กราฟจะถูกขยายด้วยลิงก์สถานะ-การกระทำไปเรื่อยๆ จนไปถึงสถานะที่เป้าหมายทั้งหมดเป็นจริงพร้อมกัน
เราถือว่าบรรลุเป้าหมายได้ หากไม่มีสถานะเป้าหมายใดๆ ในระดับปัจจุบันที่เป็นปฏิปักษ์กัน (mutually exclusive)
รายละเอียดจะอธิบายภายหลัง
<br>
Planning graph ใช้ได้เฉพาะกับปัญหาการวางแผนแบบประพจน์ (propositional) เท่านั้น ดังนั้นเราต้องกำจัดตัวแปรทั้งหมดด้วยการสร้างการแทนที่ (substitutions) ที่เป็นไปได้ทั้งหมด
<br>
ตัวอย่างเช่น planning graph ของปัญหา `have_cake_and_eat_cake_too` อาจมีหน้าตาเช่นนี้
![title](images/cake_graph.jpg)
<br>
เส้นสีดำแสดงลิงก์ระหว่างสถานะและการกระทำ
<br>
ในทุกปัญหาการวางแผน เราสามารถทำการกระทำ `no-op` ได้ กล่าวคือ เราอาจไม่เลือกการกระทำใดสำหรับสถานะหนึ่งๆ
สิ่งนี้เรียกว่า 'การคงอยู่' (Persistence) และแทนด้วยกล่องสี่เหลี่ยมเล็กๆ บนกราฟ
ในเชิงเทคนิค การกระทำแบบ persistence มีผลลัพธ์เหมือนกับเงื่อนไขก่อนของมัน
สิ่งนี้ทำให้เราสามารถพาสถานะหนึ่งไปยังระดับถัดไปได้
<br>
<br>
เส้นสีเทาแสดง “ภาวะยกเว้นร่วม” (mutual exclusivity)
หมายความว่าการกระทำที่เชื่อมด้วยเส้นสีเทาไม่สามารถทำพร้อมกันได้
ภาวะยกเว้นร่วม (mutex) เกิดขึ้นในกรณีต่อไปนี้:
1. **Inconsistent effects**: ผลของการกระทำหนึ่งขัดแย้ง/ลบล้างผลของอีกการกระทำหนึ่ง ตัวอย่างเช่น _Eat(Cake)_ และการคงอยู่ของ _Have(Cake)_ มีผลที่ไม่สอดคล้องเพราะไม่เห็นพ้องกันในเอฟเฟกต์ _Have(Cake)_
2. **Interference**: ผลอย่างหนึ่งของการกระทำหนึ่งเป็นการปฏิเสธ (negation) เงื่อนไขก่อนของอีกการกระทำ ตัวอย่างเช่น _Eat(Cake)_ แทรกแซงการคงอยู่ของ _Have(Cake)_ โดยการปฏิเสธเงื่อนไขก่อนของมัน
3. **Competing needs**: เงื่อนไขก่อนของการกระทำหนึ่งเป็น mutex กับเงื่อนไขก่อนของอีกการกระทำ ตัวอย่างเช่น _Bake(Cake)_ และ _Eat(Cake)_ เป็น mutex เพราะทั้งคู่ขัดแย้งกันที่ค่าของเงื่อนไขก่อน _Have(Cake)_

ในมอดูลนี้ ได้ติดตั้ง (implement) planning graph โดยใช้สองคลาส คือ `Level` ซึ่งเก็บข้อมูลของระดับหนึ่งๆ และ `Graph` ที่เชื่อมหลายระดับเข้าด้วยกัน
ต่อไปเรามาดูคลาส `Level` กัน

In [14]:
from planning import *
from notebook import psource

In [15]:
psource(Level)

แต่ละระดับ (level) จะเก็บข้อมูลดังนี้
1. สถานะปัจจุบันของระดับใน `current_state`
2. ลิงก์จากการกระทำไปยังเงื่อนไขก่อนของมันใน `current_action_links`
3. ลิงก์จากสถานะไปยังการกระทำที่ทำได้ในสถานะนั้นใน `current_state_links`
4. ลิงก์จากการกระทำแต่ละตัวไปยังผลของมันใน `next_action_links`
5. ลิงก์จากแต่ละการกระทำไปยังสถานะถัดไปที่เป็นไปได้ใน `next_state_links` ซึ่งจัดเก็บข้อมูลเดียวกับ `current_action_links` ของระดับถัดไป
6. ลิงก์ mutex ใน `mutex`
<br>
<br>
เมธอด `find_mutex` ใช้ค้นหาลิงก์ mutex ตามเกณฑ์ที่กล่าวไว้ข้างต้น
<br>
เมธอด `build` ใช้เติมข้อมูลโครงสร้างที่เก็บสถานะและข้อมูลการกระทำไว้
การกระทำแบบคงอยู่ (persistence) สำหรับแต่ละคลอสในสถานะปัจจุบันจะถูกนิยามที่นี่ด้วย
การกระทำแบบ persistence ที่สร้างใหม่จะใช้ชื่อเดียวกับสถานะของมัน โดยเติมคำนำหน้า 'P'

ต่อไปมาดูคลาส `Graph` กัน

In [16]:
psource(Graph)

คลาสนี้เก็บคำจำกัดความของปัญหาไว้ใน `pddl`, 
ฐานความรู้ไว้ใน `kb`, 
ลิสต์ของออบเจ็กต์ `Level` ไว้ใน `levels` และ 
อาร์กิวเมนต์ที่เป็นไปได้ทั้งหมดจากสถานะเริ่มต้นไว้ใน `objects`
<br>
เมธอด `expand_graph` ใช้สร้างระดับใหม่ของกราฟ
เมธอดนี้ถูกเรียกเมื่อเงื่อนไขเป้าหมายยังไม่เป็นจริงในระดับปัจจุบัน หรือเมื่อการกระทำที่นำไปสู่เป้าหมายเป็น mutex ต่อกัน
เมธอด `non_mutex_goals` ใช้ตรวจสอบว่าเป้าหมายในสถานะปัจจุบันไม่เป็น mutex ต่อกัน
<br>
<br>
โดยใช้สองคลาสนี้ เราสามารถนิยาม planning graph ซึ่งจะถูกใช้เพื่อให้ฮิวริสติกที่เชื่อถือได้สำหรับปัญหาการวางแผน หรือใช้ในอัลกอริทึม `GraphPlan`
<br>
ต่อไปมาดูคลาส `GraphPlan` กัน

In [17]:
psource(GraphPlan)

เมื่อได้รับปัญหาการวางแผนที่นิยามเป็น PlanningProblem แล้ว `GraphPlan` จะสร้าง planning graph ที่เก็บไว้ใน `graph` และขยายมันต่อไปจนถึงสถานะที่เป้าหมายทั้งหมดเกิดขึ้นพร้อมกันโดยไม่มีภาวะยกเว้นร่วม (mutex)
<br>
เมื่อพบเป้าหมายแล้ว จะเรียก `extract_solution`
เมธอดนี้จะค้นหาเส้นทางไปยังคำตอบแบบเวียนเกิดซ้ำโดยอิงจาก planning graph
ในกรณีที่ `extract_solution` หาเส้นทางไม่ได้สำหรับชุดเป้าหมาย ณ ระดับหนึ่งๆ เราจะบันทึกคู่ `(level, goals)` เป็น **no-good**
เมื่อใดก็ตามที่ `extract_solution` ถูกเรียกอีกครั้งด้วยระดับและเป้าหมายเดียวกัน เราจะพบ no-good ที่บันทึกไว้และคืนค่าความล้มเหลวทันที แทนที่จะค้นหาใหม่
no-good ยังถูกใช้ในเงื่อนไขหยุด (termination test) ด้วย
<br>
เมธอด `check_leveloff` ใช้ตรวจสอบว่า planning graph สำหรับปัญหานั้น **หยุดคงที่** (levelled-off) แล้วหรือไม่ กล่าวคือ มีสถานะ การกระทำ และคู่ mutex เหมือนเดิมกับระดับก่อนหน้า
ถ้ากราฟหยุดคงที่แล้วและเรายังไม่พบคำตอบ ก็ไม่ควรขยายกราฟต่อ เพราะจะไม่เกิดข้อมูลใหม่
ในกรณีเช่นนี้ เราสามารถสรุปได้ว่าปัญหาการวางแผนแก้ไม่ได้ภายใต้ข้อจำกัดที่กำหนด
<br>
<br>
สรุปแล้ว อัลกอริทึม `GraphPlan` จะเรียก `expand_graph` และตรวจสอบว่าได้ถึงเป้าหมายแล้วหรือไม่ และว่าเป้าหมายไม่เป็น mutex
<br>
ถ้าใช่ จะเรียก `extract_solution` ซึ่งสร้างคำตอบกลับจาก planning graph แบบเวียนเกิดซ้ำ
<br>
ถ้าไม่ใช่ เราจะตรวจสอบว่ากราฟหยุดคงที่หรือยัง และดำเนินการต่อหากยังไม่หยุด

ต่อไปเรามาลองแก้ปัญหาการวางแผนบางปัญหาที่เราได้กำหนดไว้ก่อนหน้านี้กัน

#### ปัญหาขนส่งทางอากาศ (Air cargo problem)
ตามสรุปด้านบน เราได้กำหนดฟังก์ชันผู้ช่วยเพื่อเรียกใช้ `GraphPlan` กับปัญหา `air_cargo`
ฟังก์ชันค่อนข้างตรงไปตรงมา
ลองมาดูกัน

In [18]:
psource(air_cargo_graphplan)

มาสร้างอินสแตนซ์ของปัญหาและหาคำตอบโดยใช้ฟังก์ชันผู้ช่วยนี้กัน

In [19]:
airCargoG = air_cargo_graphplan()
airCargoG

[[[Load(C2, P2, JFK),
   PAirport(SFO),
   PAirport(JFK),
   PPlane(P2),
   PPlane(P1),
   Fly(P2, JFK, SFO),
   PCargo(C2),
   Load(C1, P1, SFO),
   Fly(P1, SFO, JFK),
   PCargo(C1)],
  [Unload(C2, P2, SFO), Unload(C1, P1, JFK)]]]

แต่ละองค์ประกอบในคำตอบคือการกระทำที่ถูกต้อง
คำตอบถูกแยกเป็นลิสต์ตามแต่ละระดับ
การกระทำที่ขึ้นต้นด้วย 'P' คือการกระทำแบบคงอยู่ (persistence) และสามารถละไว้ได้
มันเพียงแค่พาสถานะบางอย่างไปยังระดับถัดไป
เรายังมีฟังก์ชันผู้ช่วย `linearize` ที่แสดงคำตอบให้อ่านง่ายขึ้น คล้ายกับตัววางแผนแบบลำดับรวม (total-order planner) แต่ย้ำว่าเครื่องมือนี้ _ไม่ใช่_ ตัววางแผนแบบลำดับรวม

In [20]:
linearize(airCargoG)

[Load(C2, P2, JFK),
 Fly(P2, JFK, SFO),
 Load(C1, P1, SFO),
 Fly(P1, SFO, JFK),
 Unload(C2, P2, SFO),
 Unload(C1, P1, JFK)]

ถูกต้องแล้ว นี่คือคำตอบที่ถูกต้อง
<br>
เรายังมีฟังก์ชันผู้ช่วยในลักษณะเดียวกันสำหรับปัญหาการวางแผนอื่นๆ บางปัญหา
<br>
ลองแก้ปัญหา spare tire กัน

In [21]:
spareTireG = spare_tire_graphplan()
linearize(spareTireG)

[Remove(Spare, Trunk), Remove(Flat, Axle), PutOn(Spare, Axle)]

คำตอบสำหรับปัญหาเค้ก (cake problem)

In [22]:
cakeProblemG = have_cake_and_eat_cake_too_graphplan()
linearize(cakeProblemG)

[Eat(Cake), Bake(Cake)]

คำตอบสำหรับการตั้งค่า Sussman's Anomaly ของบล็อกสามก้อน

In [23]:
sussmanAnomalyG = three_block_tower_graphplan()
linearize(sussmanAnomalyG)

[MoveToTable(C, A), Move(B, Table, C), Move(A, Table, B)]

คำตอบของปัญหาถุงเท้าและรองเท้า (socks and shoes)

In [24]:
socksShoesG = socks_and_shoes_graphplan()
linearize(socksShoesG)

[RightSock, LeftSock, RightShoe, LeftShoe]